## Packages

In [ ]:
import nltk
import nltk
nltk.download('punkt')
import re 
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import pandas as pd

import os

import json

from nltk.corpus import stopwords
import lemmy

## Data

In [ ]:
corpus = pd.read_pickle("virksomhedsoverdragelse_samlet.pkl") # Note: data is not available

## Filter -  Document related to tender with Employee transfers

### Recode filename

In [2]:
def cleanfilename(filename):
    filename = re.sub('[-_,]', '', filename) # removes special characters and symbols from title
    filename = re.sub('[.]', ' ', filename) # Recode period to space
    return filename

In [ ]:
corpus['filename'] = corpus.filename.apply(lambda x : cleanfilename(x))
corpus.head(20)

### Identify document in which employee transfers is included based on document ID

In [5]:
def extract_filter(filename):
    FileObj = open(filename, 'rb')
    read = pd.read_excel(filename, engine='openpyxl')
    read = read.to_string(index=False)
    return (read)

In [6]:
filter_doc = extract_filter (r'doc_id virksomhedsoverdragelser.xlsx') # Note: data is not available

In [7]:
filter_doc = filter_doc.split("\n")

In [8]:
filter_doc = [s.replace("-", "")for s in filter_doc]
filter_doc = [s.replace("_", "")for s in filter_doc]

In [ ]:
filter_doc = [x.strip(' ') for x in filter_doc]

In [ ]:
corpus = corpus[corpus["filename"].str.contains('|'.join(filter_doc))]

In [18]:
corpus.head(20)

,filename,text,filedir,clean_text,word,lemmy,virksomhedsoverdragelse
15,Udbudsresume 3672982021 pdf,"Danmark-København: Reparation, vedligeholdelse...",F:\contract-skills-and-performance\Download af...,danmark københavn reparation vedligeholdelse o...,"[danmark, københavn, reparation, vedligeholdel...","[danmark, københavn, reparation, vedligeholdel...",[]
151,Udbudsresume 3672982021 pdf,"Danmark-København: Reparation, vedligeholdelse...",F:\contract-skills-and-performance\Download af...,danmark københavn reparation vedligeholdelse o...,"[danmark, københavn, reparation, vedligeholdel...","[danmark, københavn, reparation, vedligeholdel...",[]
5273,Erhvervslejekontrakt Bilag 1 plantegning ove...,,F:\contract-skills-and-performance\Download af...,,[],[],[]
5274,Erhvervslejekontrakt Bilag 2 stueplan 129286...,5-592\r\n\r\nCarolineparken - leje af lokaler\...,F:\contract-skills-and-performance\Download af...,5 592 carolineparken leje af lokaler storkøkke...,"[5, 592, carolineparken, leje, af, lokaler, st...","[5, 592, carolinepark, leje, af, lokale, stork...",[]
5275,Erhvervslejekontrakt Bilag 3 kælderfacilitet...,5-592\r\n\r\nCarolineparken - leje af lokaler\...,F:\contract-skills-and-performance\Download af...,5 592 carolineparken leje af lokaler storkøkke...,"[5, 592, carolineparken, leje, af, lokaler, st...","[5, 592, carolinepark, leje, af, lokale, stork...",[]
5276,Erhvervslejekontrakt Bilag 4 Fotooversigt 129...,,F:\contract-skills-and-performance\Download af...,,[],[],[]
5277,Erhvervslejekontrakt Bilag 5 Inventarliste o...,Inventarliste pr. m...,F:\contract-skills-and-performance\Download af...,inventarliste pr marts 2021 unnamed 1 unnamed...,"[inventarliste, pr, marts, 2021, unnamed, 1, u...","[inventarliste, per, marts, 2021, unnamed, 1, ...",[]
5278,Erhvervslejekontrakt Bilag 6 lejeberegning 1...,Bilag 6\r\nErhvervslejekontrakt\r\n\r\nDok.nr....,F:\contract-skills-and-performance\Download af...,bilag 6 erhvervslejekontrakt dok nr 20318 21 s...,"[bilag, 6, erhvervslejekontrakt, dok, nr, 2031...","[bilag, 6, erhvervslejekontrakt, dokke, nr, 20...",[]
5279,Erhvervslejekontrakt Byog Boligministeriets c...,Tjekliste - Erhvervslejeloven\r\nDenne vejledn...,F:\contract-skills-and-performance\Download af...,tjekliste erhvervslejeloven denne vejledning e...,"[tjekliste, erhvervslejeloven, denne, vejledni...","[tjekliste, erhvervslejelov, denne, vejledning...",[]
5280,Kontrakt Delaftale 1 Madservice Plejecentren...,Kontrakt Madservice \n\nDelaftale 1 - Plejecen...,F:\contract-skills-and-performance\Download af...,kontrakt madservice delaftale 1 plejecentre 23...,"[kontrakt, madservice, delaftale, 1, plejecent...","[kontrakt, madservice, delaftale, 1, plejecent...","[virksomhedsoverdragelse, virksomhedsoverdrage..."


## Enables searching for uppercase letters

In [6]:
def cleanSTOR(text):
    text = re.sub('\s\s+', ' ', str(text)) # add spaces at line breaks
    text = re.sub("\n", "", text) # remove line breaks
    text = re.sub("\r", " ", text) # remove line breaks in doc
    text = re.sub("[()]","", text) # remove parentheses
    text = re.sub("%", "", text) # remove percentage
    text = re.sub('[-()\"#/@;:<>{}`+=~|.!?,‰]', ' ', text) # remove special characters and symbols
    text = re.sub(' +', ' ', text) # remove too many spaces
    return text

In [7]:
corpus['clean_stor'] = corpus.text.apply(lambda x : cleanSTOR(x))

In [8]:
corpus.head()

,filename,text,filedir,clean_text,word,lemmy,virksomhedsoverdragelse_2022_filter,clean_stor
15,Udbudsresume 3672982021 pdf,"Danmark-København: Reparation, vedligeholdelse...",F:\contract-skills-and-performance\Download af...,danmark københavn reparation vedligeholdelse o...,"[danmark, københavn, reparation, vedligeholdel...","[danmark, københavn, reparation, vedligeholdel...",[],Danmark København Reparation vedligeholdelse o...
151,Udbudsresume 3672982021 pdf,"Danmark-København: Reparation, vedligeholdelse...",F:\contract-skills-and-performance\Download af...,danmark københavn reparation vedligeholdelse o...,"[danmark, københavn, reparation, vedligeholdel...","[danmark, københavn, reparation, vedligeholdel...",[],Danmark København Reparation vedligeholdelse o...
5273,Erhvervslejekontrakt Bilag 1 plantegning ove...,,F:\contract-skills-and-performance\Download af...,,[],[],[],
5274,Erhvervslejekontrakt Bilag 2 stueplan 129286...,5-592\r\n\r\nCarolineparken - leje af lokaler\...,F:\contract-skills-and-performance\Download af...,5 592 carolineparken leje af lokaler storkøkke...,"[5, 592, carolineparken, leje, af, lokaler, st...","[5, 592, carolinepark, leje, af, lokale, stork...",[],5 592 Carolineparken leje af lokaler Storkøkke...
5275,Erhvervslejekontrakt Bilag 3 kælderfacilitet...,5-592\r\n\r\nCarolineparken - leje af lokaler\...,F:\contract-skills-and-performance\Download af...,5 592 carolineparken leje af lokaler storkøkke...,"[5, 592, carolineparken, leje, af, lokaler, st...","[5, 592, carolinepark, leje, af, lokale, stork...",[],5 592 Carolineparken leje af lokaler Storkøkke...


## Define Safeguards 

Note that numeration is not the same as in the article.

In [9]:
regel_2 = ['ansættelsespapir', 'ansættelsesbrev', 'arbejdsbrev', 'ansættelseskontrakter', 'ansættelsesbevis', 'ansættelsesbevislov'] 

In [10]:
def Filter2(lemmy, regel_2):
    return [n for n in lemmy if
             any(m in n for m in regel_2)]

In [11]:
corpus['tema_2'] = corpus.lemmy.apply(lambda x, y=regel_2 : Filter2(x, y))

In [17]:
regel_3_1 = ['medarbejderrepræsentant', 'arbejdsmiljørepræsentant']  

In [13]:
regel_3_2 = [' AMR ', ' TRIO ']

In [19]:
def Filter3_1(lemmy, regel_3_1):
    return [n for n in lemmy if
             any(m in n for m in regel_3_1)]

In [15]:
def Filter3_2(clean_stor, regel_3_2):
    regel_3_2 = ('|'.join(regel_3_2))
    clean_stor = re.findall(regel_3_2, clean_stor)
    return clean_stor

In [20]:
corpus['tema_3_1'] = corpus.lemmy.apply(lambda x, y=regel_3_1 : Filter3_1(x, y))

In [21]:
corpus['tema_3_2'] = corpus.clean_stor.apply(lambda x, y=regel_3_2 : Filter3_2(x, y))

In [159]:
corpus['tema_3'] = corpus['tema_3_1'] + corpus['tema_3_2']

In [180]:
regel_4_1 = ['med udvalg', 'medudvalg', "medudvalget", 'med udvalget', 'rammeaftale om medindflydelse og medbestemmelse','direktiv 299214EF', 'direktiv 2992 14 EF']

In [181]:
def Filter4_1(clean_text, regel_4_1):
    regel_4_1 = ('|'.join(regel_4_1))
    clean_text = re.findall(regel_4_1, clean_text)
    return clean_text

In [25]:
regel_4_2 = ['medindflyd', 'medarbejderhåndbog']

In [26]:
def Filter4_2(lemmy, regel_4_2):
    return [n for n in lemmy if
             any(m in n for m in regel_4_2)]

In [179]:
regel_4_3 = ['MED rammeaftale', 'MED rammeaftalen', 'MEDrammeaftale', 'MEDrammeaftalen']

In [183]:
def Filter4_3(clean_stor, regel_4_3):
    regel_4_3 = ('|'.join(regel_4_3))
    clean_stor = re.findall(regel_4_3, clean_stor)
    return clean_stor

In [182]:
corpus['tema_4_1'] = corpus.clean_text.apply(lambda x, y=regel_4_1 : Filter4_1(x, y))

In [28]:
corpus['tema_4_2'] = corpus.lemmy.apply(lambda x, y=regel_4_2 : Filter4_2(x, y))

In [184]:
corpus['tema_4_3'] = corpus.clean_stor.apply(lambda x, y=regel_4_3 : Filter4_3(x, y))

In [185]:
corpus['tema_4'] = corpus['tema_4_1'] + corpus['tema_4_2'] + corpus['tema_4_3']

In [30]:
regel_5 = ['arbejdsmiljøloven', 'arbejdstilsyn']

In [31]:
def Filter5(lemmy, regel_5):
    return [n for n in lemmy if
             any(m in n for m in regel_5)]

In [32]:
corpus['tema_5'] = corpus.lemmy.apply(lambda x, y=regel_5 : Filter5(x, y))

In [33]:
regel_6 = ['løn og ansættelsesforhold', 'løn og ansættelsesforholdet', 'løn og arbejdsvilkår']

In [34]:
def Filter6(clean_text, regel_6):
    regel_6 = ('|'.join(regel_6))
    clean_text = re.findall(regel_6, clean_text)
    return clean_text

In [35]:
corpus['tema_6'] = corpus.clean_text.apply(lambda x, y=regel_6 : Filter6(x, y))

In [37]:
regel_7_1 = ['ensidigt gentaget arbejde', 'tung løft']

In [41]:
def Filter7_1(clean_text, regel_7_1):
    regel_7_1 = ('|'.join(regel_7_1))
    clean_text = re.findall(regel_7_1, clean_text)
    return clean_text

In [36]:
regel_7_2 = ['sygefravær', 'arbejdsulykke', 'arbejdstid', 'mobning', 'overbelastning', 'udbrænd', 'arbejdstempo', 'arbejdstakt']

In [41]:
def Filter7_2(lemmy, regel_7_2):
    return [n for n in lemmy if
             any(m in n for m in regel_7_2)]

In [38]:
regel_7_3 = [' EGA ']

In [40]:
def Filter7_3(clean_stor, regel_7_3):
    regel_7_3 = ('|'.join(regel_7_3))
    clean_stor = re.findall(regel_7_3, clean_stor)
    return clean_stor

In [43]:
corpus['tema_7_1'] = corpus.clean_text.apply(lambda x, y=regel_7_1 : Filter7_1(x, y))

In [44]:
corpus['tema_7_2'] = corpus.lemmy.apply(lambda x, y=regel_7_2 : Filter7_2(x, y))

In [45]:
corpus['tema_7_3'] = corpus.clean_stor.apply(lambda x, y=regel_7_3 : Filter7_3(x, y))

In [46]:
corpus['tema_7'] = corpus['tema_7_1'] + corpus['tema_7_2'] + corpus['tema_7_3']

In [47]:
regel_8_1 = ['kronesmiley', 'krone smiley', 'grøn smiley']

In [48]:
regel_8_2 = ['arbejdsmiljøledelse', 'miljøvejvis', 'arbejdsmiljøcertificering']

In [49]:
def Filter8_1(clean_text, regel_8_1):
    regel_8_1 = ('|'.join(regel_8_1))
    clean_text = re.findall(regel_8_1, clean_text)
    return clean_text

In [50]:
def Filter8_2(lemmy, regel_8_2):
    return [n for n in lemmy if
             any(m in n for m in regel_8_2)]

In [51]:
corpus['tema_8_1'] = corpus.clean_text.apply(lambda x, y=regel_8_1 : Filter8_1(x, y))

In [52]:
corpus['tema_8_2'] = corpus.lemmy.apply(lambda x, y=regel_8_2 : Filter8_2(x, y))

In [53]:
corpus['tema_8'] = corpus['tema_8_1'] + corpus['tema_8_2']

In [54]:
regel_10_1 = ['social klausul', 'sociale klausuler', 'social ansvar', 'socialt ansvar', 'socialt medansvar', 'global compact', ' etik ', ' etisk ']

In [55]:
regel_10_2 = ['ligestilling', 'diskrimination', 'samfundsansvar', 'etnisk', 'menneskerettighed' ]

In [56]:
def Filter10_1(clean_text, regel_10_1):
    regel_10_1 = ('|'.join(regel_10_1))
    clean_text = re.findall(regel_10_1, clean_text)
    return clean_text

In [57]:
def Filter10_2(lemmy, regel_10_2):
    return [n for n in lemmy if
             any(m in n for m in regel_10_2)]

In [58]:
corpus['tema_10_1'] = corpus.clean_text.apply(lambda x, y=regel_10_1 : Filter10_1(x, y))

In [59]:
corpus['tema_10_2'] = corpus.lemmy.apply(lambda x, y=regel_10_2 : Filter10_2(x, y))

In [60]:
corpus['tema_10'] = corpus['tema_10_1'] + corpus['tema_10_2']

In [61]:
regel_11 = ['fastholdelsespolitik', 'rekruttering']

In [62]:
def Filter11(lemmy, regel_11):
    return [n for n in lemmy if
             any(m in n for m in regel_11)]

In [63]:
corpus['tema_11'] = corpus.lemmy.apply(lambda x, y=regel_11 : Filter11(x, y))

In [9]:
regel_12 = ['uddannelsesklausul', 'videreuddannelse', 'mentor', 'uddannelsespolitik', 'kompetenceudvikling', 'opkvalificer', 'efteruddannelse', 'uddannelseskrav', 'uddannelsesaftale']

In [10]:
def Filter12(lemmy, regel_12):
    return [n for n in lemmy if
             any(m in n for m in regel_12)]

In [11]:
corpus['tema_12'] = corpus.lemmy.apply(lambda x, y=regel_12 : Filter12(x, y))

In [67]:
regel_13_1 = [' MUS ']

In [68]:
regel_13_2 = ['medarbejderudviklingssamtal']

In [69]:
def Filter13_1(clean_stor, regel_13_1):
    regel_13_1 = ('|'.join(regel_13_1))
    clean_stor = re.findall(regel_13_1, clean_stor)
    return clean_stor

In [70]:
def Filter13_2(lemmy, regel_13_2):
    return [n for n in lemmy if
             any(m in n for m in regel_13_2)]

In [71]:
corpus['tema_13_1'] = corpus.clean_stor.apply(lambda x, y=regel_13_1 : Filter13_1(x, y))

In [72]:
corpus['tema_13_2'] = corpus.lemmy.apply(lambda x, y=regel_13_2 : Filter13_2(x, y))

In [73]:
corpus['tema_13'] = corpus['tema_13_1'] + corpus['tema_13_2']

In [74]:
regel_15 = ['tvangsarbejde', 'ilokonvention nr 29', 'ilo konvention nr 29', 'ilokonvention nr 105', 'ilo konvention nr 105']

In [75]:
def Filter15(clean_text, regel_15):
    regel_15 = ('|'.join(regel_15))
    clean_text = re.findall(regel_15, clean_text)
    return clean_text

In [76]:
corpus['tema_15'] = corpus.clean_text.apply(lambda x, y=regel_15 : Filter15(x, y))

In [77]:
regel_16 = ['kontrakthavers pligt til at sikre dennes ansattes vilkår', 'ilokonventionen nr 94', 'ilo konventionen nr 94']

In [78]:
def Filter16(clean_text, regel_16):
    regel_16 = ('|'.join(regel_16))
    clean_text = re.findall(regel_16, clean_text)
    return clean_text

In [79]:
corpus['tema_16'] = corpus.clean_text.apply(lambda x, y=regel_16 : Filter16(x, y))

In [80]:
regel_17 = ['ingen diskrimination i ansættelsen', 'ilokonvention nr 100', 'ilo konvention nr 100', 'ilokonvention nr 111', 'ilo konvention nr 111']

In [81]:
def Filter17(clean_text, regel_17):
    regel_17 = ('|'.join(regel_17))
    clean_text = re.findall(regel_17, clean_text)
    return clean_text

In [82]:
corpus['tema_17'] = corpus.clean_text.apply(lambda x, y=regel_17 : Filter17(x, y))

In [83]:
regel_18 = ['mindstealder for adgang til beskæftigelse samt forbud mod og omgående indsats til afskaffelse af de værste former for børnearbejde', 'ilokonvention nr 138', 'ilo konvention nr 138', 'ilokonvention nr 182', 'ilo konvention nr 182']

In [84]:
def Filter18(clean_text, regel_18):
    regel_18 = ('|'.join(regel_18))
    clean_text = re.findall(regel_18, clean_text)
    return clean_text

In [85]:
corpus['tema_18'] = corpus.clean_text.apply(lambda x, y=regel_18 : Filter18(x, y))

In [86]:
regel_19 = ['sikkert og sundt arbejdsmiljø', 'ilokonvention nr 155', 'ilo konvention nr 155']

In [87]:
def Filter19(clean_text, regel_19):
    regel_19 = ('|'.join(regel_19))
    clean_text = re.findall(regel_19, clean_text)
    return clean_text

In [88]:
corpus['tema_19'] = corpus.clean_text.apply(lambda x, y=regel_19 : Filter19(x, y))

In [89]:
regel_20 = ['organisationsfrihed og ret til kollektive forhandlinger', 'ilokonvention nr 87', 'ilo konvention nr 87', 'ilokonvention nr 98', 'ilo konvention nr 98', 'ilokonvention nr 135', 'ilo konvention nr 135']

In [90]:
def Filter20(clean_text, regel_20):
    regel_20 = ('|'.join(regel_20))
    clean_text = re.findall(regel_20, clean_text)
    return clean_text

In [91]:
corpus['tema_20'] = corpus.clean_text.apply(lambda x, y=regel_20 : Filter20(x, y))

In [92]:
regel_22 = ['liste med medarbejdere', 'oversigt over medarbejde', 'medarbejderoversigt', 'medarbejderoversigter', 'medarbejderoversigten']

In [93]:
def Filter22(clean_text, regel_22):
    regel_22 = ('|'.join(regel_22))
    clean_text = re.findall(regel_22, clean_text)
    return clean_text

In [94]:
corpus['tema_22'] = corpus.clean_text.apply(lambda x, y=regel_22 : Filter22(x, y))

In [95]:
regel_23_1 = ['forsikring af medarbejde', 'forsikring af ansatte', 'forsikring af person']

In [96]:
regel_23_2 = ['arbejdsskadeforsikring']

In [97]:
def Filter23_1(clean_text, regel_23_1):
    regel_23_1 = ('|'.join(regel_23_1))
    clean_text = re.findall(regel_23_1, clean_text)
    return clean_text

In [98]:
def Filter23_2(lemmy, regel_23_2):
    return [n for n in lemmy if
             any(m in n for m in regel_23_2)]

In [99]:
corpus['tema_23_1'] = corpus.clean_text.apply(lambda x, y=regel_23_1 : Filter23_1(x, y))

In [100]:
corpus['tema_23_2'] = corpus.lemmy.apply(lambda x, y=regel_23_2 : Filter23_2(x, y))

In [101]:
corpus['tema_23'] = corpus['tema_23_1'] + corpus['tema_23_2']

In [187]:
regel_27_1 = ['3F', 'FOA', 'HK', 'OAO', ' FH ']

In [131]:
regel_27_2 = ['fagforening', 'fagforeningen', 'fagforeninger', 'dansk metal', 'ny udbudsmodel', 'treiét udbud', 'treiet udbud', 'tre i ét udbud', 'tre i et udbud', 'faglige organisation', 'faglige organisationer', 'faglig organisation', 'faglig organisatoner']

In [188]:
def Filter27_1(clean_stor, regel_27_1):
    regel_27_1 = ('|'.join(regel_27_1))
    clean_stor = re.findall(regel_27_1, clean_stor)
    return clean_stor

In [133]:
def Filter27_2(clean_text, regel_27_2):
    regel_27_2 = ('|'.join(regel_27_2))
    clean_text = re.findall(regel_27_2, clean_text)
    return clean_text

In [189]:
corpus['tema_27_1'] = corpus.clean_stor.apply(lambda x, y=regel_27_1 : Filter27_1(x, y))

In [137]:
corpus['tema_27_2'] = corpus.clean_text.apply(lambda x, y=regel_27_2 : Filter27_2(x, y))

In [190]:
corpus['tema_27'] = corpus['tema_27_1'] + corpus['tema_27_2']

In [143]:
regel_28_1 = [' RUT ']

In [139]:
regel_28_2 = ['kædeansvar']

In [140]:
def Filter28_1(clean_stor, regel_28_1):
    regel_28_1 = ('|'.join(regel_28_1))
    clean_stor = re.findall(regel_28_1, clean_stor)
    return clean_stor

In [141]:
def Filter28_2(lemmy, regel_28_2):
    return [n for n in lemmy if
             any(m in n for m in regel_28_2)]

In [144]:
corpus['tema_28_1'] = corpus.clean_stor.apply(lambda x, y=regel_28_1 : Filter28_1(x, y))

In [145]:
corpus['tema_28_2'] = corpus.lemmy.apply(lambda x, y=regel_28_2 : Filter28_2(x, y))

In [146]:
corpus['tema_28'] = corpus['tema_28_1'] + corpus['tema_28_2']

In [147]:
regel_29 = ['lønrefusion', 'medarbejdersrefusion', 'medarbejdsrefusion']

In [148]:
def Filter29(lemmy, regel_29):
    return [n for n in lemmy if
             any(m in n for m in regel_29)]

In [149]:
corpus['tema_29'] = corpus.lemmy.apply(lambda x, y=regel_29 : Filter29(x, y))

In [150]:
regel_33_1 = ['rekrutteringspolitik', 'afskedigelsespolitik', 'udviklingspolitik', 'mobbepolitik', 'stresspolitik', 'seniorpolitik', 'personalepolitik']

In [151]:
regel_33_2 = ['hr politik', 'hrpolitik']

In [152]:
def Filter33_1(lemmy, regel_33_1):
    return [n for n in lemmy if
             any(m in n for m in regel_33_1)]

In [155]:
def Filter33_2(clean_text, regel_33_2):
    regel_33_2 = ('|'.join(regel_33_2))
    clean_text = re.findall(regel_33_2, clean_text)
    return clean_text

In [153]:
corpus['tema_33_1'] = corpus.lemmy.apply(lambda x, y=regel_33_1 : Filter33_1(x, y))

In [156]:
corpus['tema_33_2'] = corpus.clean_text.apply(lambda x, y=regel_33_2 : Filter33_2(x, y))

In [158]:
corpus['tema_33'] = corpus['tema_33_1'] + corpus['tema_33_2']

In [170]:
regel_35 = ['arbejdsklausul', 'arbejdsmiljøklausul']

In [172]:
def Filter35(lemmy, regel_35):
    return [n for n in lemmy if
             any(m in n for m in regel_35)]

In [173]:
corpus['tema_35'] = corpus.lemmy.apply(lambda x, y=regel_35 : Filter35(x, y))

In [5]:
regel_36 = ['samarbejdsudvalg']

In [6]:
def Filter36(lemmy, regel_36):
    return [n for n in lemmy if
             any(m in n for m in regel_36)]

In [7]:
corpus['tema_36'] = corpus.lemmy.apply(lambda x, y=regel_36 : Filter36(x, y))

## Save and export

In [174]:
corpus.to_pickle("virksomhedsoverdragelser_safeguards.pkl")

In [175]:
corpus.to_csv("virksomhedsoverdragelser_tema_version3.csv", mode = 'w', columns=['filename', 'filedir', 'tema_2', 'tema_3', 'tema_4', 'tema_5', 'tema_6', 'tema_7', 'tema_8', 'tema_10', 'tema_11', 'tema_12', 'tema_13', 'tema_15', 'tema_16', 'tema_17', 'tema_18', 'tema_19', 'tema_20', 'tema_22', 'tema_23', 'tema_27', 'tema_28', 'tema_29', 'tema_33', 'tema_35', 'tema_36'], index=False)